# MultiCoNER 2 - Few-Shot NER with Gemini API (Optimized + Full Prompt)

## Optimizations Applied:
- **Batch Processing**: 10 examples per API call (10x faster)
- **Structured JSON Output**: No parsing errors
- **Original Full Prompt**: Keeping your detailed 2,500 token prompt for robustness
- **Expected Time**: ~2 hours for 10K examples (vs 16+ hours before)
- **Expected Cost**: ~$80-100 (vs $574 before)

## 1. Setup and Imports

In [ ]:
# Import required libraries
import google.generativeai as genai
from google.colab import userdata
import pandas as pd
import json
import time
from tqdm.notebook import tqdm

print("✓ All libraries imported successfully.")

## 2. Configure Gemini API with Structured Output

In [ ]:
# Get API key from Colab secrets
GOOGLE_API_KEY = userdata.get('gemini_api_key')
genai.configure(api_key=GOOGLE_API_KEY)

# Configure for structured JSON output (prevents parsing errors)
generation_config = {
    "response_mime_type": "application/json",
}

# Initialize Gemini model with structured output
gemini_model = genai.GenerativeModel(
    'gemini-2.5-flash',
    generation_config=generation_config
)

print("✓ Gemini API configured with structured JSON output")
print(f"  Model: {gemini_model._model_name}")

## 3. Full System Prompt (Original Detailed Version)

In [ ]:
FULL_SYSTEM_PROMPT = """### SYSTEM INSTRUCTION
**MODE: ACCURACY-FIRST / RESEARCH-ENABLED**
You are a slow, deliberate reasoning agent. You must NOT guess.
1. **Scan**: Identify every proper noun or capitalized token in the input.
2. **Verify**: If you are not 100% sure of an entity's type (e.g., is "Finisterre" a place or a book?), you MUST pause and search the internet.
3. **Classify**: Apply the strict class definitions below based on your search results.
4. **Format**: Output only the final JSON list.

---

### Role
You are an expert linguist and data labeling specialist specifically trained for the MultiCoNER 2 Shared Task. You possess deep knowledge of fine-grained Named Entity Recognition (NER). **Crucially, you act as a researcher who verifies facts using the internet when faced with ambiguity.**

### Context
The user will provide a list of text tokens (words/sub-words) derived from search queries, social media, or noisy web text. Your task is to analyze these tokens and map each one to a specific Named Entity Recognition tag. The data contains ambiguity, typos (e.g., "united stats"), and lacks capitalization cues.

### Rules
1. **Input**: A JSON list of tokens (e.g., `["new", "york", "is", "big"]`).
2. **Task**: Assign a BIO (Begin, Inside, Outside) tag to every single token.
3. **Classes**: You must strictly use only the following 7 entity categories:
   - **Artist**: Musicians, bands, actors, authors, directors, painters. (e.g., "simon mayo", "picasso")
   - **Politician**: Government officials, politicians, heads of state. (e.g., "obama", "frank d. o'connor")
   - **HumanSettlement**: Cities, towns, villages, states, countries, counties. (e.g., "busan", "cleveland", "ohio")
   - **PublicCorp**: Commercial companies, businesses, brands. (e.g., "safeway", "mcdonald 's", "s&p global ratings")
   - **ORG**: Non-commercial organizations, government agencies, political parties, sports teams, unions. (e.g., "democrat", "united stats census bureau", "real madrid")
   - **Facility**: Buildings, stadiums, airports, highways, public places. (e.g., "village hall", "lanxess arena")
   - **OtherPER**: Persons who are not artists or politicians (e.g., athletes, scientists, soldiers, fictional characters, or general people). (e.g., "zcrny", "peter bourne")
   - **O**: Tokens that are not part of a named entity (CRITICAL: This includes Books, Movies, Songs, Albums, and Products).
4. **Tagging Scheme**:
   - Use `B-<Category>` for the first token of an entity.
   - Use `I-<Category>` for all subsequent tokens of the same entity.
   - Use `O` for non-entities.

### Verification Strategy (CRITICAL)
**If you are unsure about a proper noun, you MUST SEARCH THE INTERNET.**
* **Ambiguity**: If a word looks like a name (e.g., "finisterre", "wclv", "zcrny") but you don't know it, pause and search for it.
* **Distinction Logic**:
    * If search shows it is a **Book, Movie, Album, or Product** -> Tag as **O**. (We do not have tags for these in this specific task).
    * If search shows it is a **Company** -> Check if it is commercial (**PublicCorp**) or non-profit/sports (**ORG**).
    * If search shows it is a **Person** -> Check if they are a politician (**Politician**), creator (**Artist**), or other (**OtherPER**).

### Constraints
1. **Length Consistency**: The output list MUST have exactly the same number of items as the input list.
2. **Format**: Output ONLY a raw JSON list of strings. Do not include markdown formatting, explanations, or notes.
3. **Robustness**: Treat lower-cased proper nouns as entities (e.g., "paris" -> B-HumanSettlement). Context is king.

### Examples
**Input**:
["frank", "d.", "o'connor", "(", "1909", "–", "1992", ")", "lawyer", "judge", "and", "politician", "−", "head", "trauma"]
**Output**:
["B-Politician", "I-Politician", "I-Politician", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]

**Input**:
["prior", "to", "the", "stabbings", "he", "was", "an", "employee", "of", "safeway", "."]
**Output**:
["O", "O", "O", "O", "O", "O", "O", "O", "O", "B-PublicCorp", "O"]

**Input**:
["its", "current", "representative", "is", "democrat", "bruce", "antone", "."]
**Output**:
["O", "O", "O", "O", "B-ORG", "B-Politician", "I-Politician", "O"]

**Input**:
["finisterre", "a", "1943", "poetry", "collection", "by", "eugenio", "montale"]
**Output**:
["O", "O", "O", "O", "O", "O", "B-Artist", "I-Artist"]
*(Note: Search reveals 'Finisterre' is a book, so it is tagged 'O')*

---

### Batch Processing Instructions
I will provide multiple examples as a JSON object with numbered keys ("0", "1", "2", etc.).
Each value is a list of tokens to tag.

You must return a JSON object with the SAME keys, where each value is a list of BIO tags corresponding to the input tokens.

**Example Batch Input**:
{
  "0": ["frank", "d.", "o'connor"],
  "1": ["safeway", "is", "big"],
  "2": ["he", "lives", "in", "paris"]
}

**Example Batch Output**:
{
  "0": ["B-Politician", "I-Politician", "I-Politician"],
  "1": ["B-PublicCorp", "O", "O"],
  "2": ["O", "O", "O", "B-HumanSettlement"]
}
"""

print("✓ Full system prompt defined")
print(f"  Prompt length: {len(FULL_SYSTEM_PROMPT)} characters (~2,500 tokens)")
print(f"  Includes: System instructions, verification strategy, 4 examples + batch instructions")

## 4. Batch Prediction Function (10x Faster!)

In [ ]:
def get_batch_predictions(model, system_prompt, batch_tokens_list, delay_seconds=2.0):
    """
    Process multiple examples in a single API call.
    
    Args:
        model: The initialized Gemini model
        system_prompt: The system prompt with instructions
        batch_tokens_list: List of token lists, e.g.:
            [["frank", "d.", "o'connor"], ["he", "is", "from", "busan"], ...]
        delay_seconds: Delay before API call (default 2.0 for batching)
    
    Returns:
        List of predicted tag lists, or error strings for failed examples
    """
    # Add delay to avoid rate limiting
    time.sleep(delay_seconds)
    
    # Format batch as numbered JSON object
    batch_input = {
        str(i): tokens 
        for i, tokens in enumerate(batch_tokens_list)
    }
    
    # Construct prompt
    final_prompt = f"""{system_prompt}

Process these {len(batch_tokens_list)} examples:

Input:
{json.dumps(batch_input, indent=2)}

Output (JSON object with same keys):
"""
    
    try:
        # Call Gemini API
        response = model.generate_content(final_prompt)
        raw_response = response.text.strip()
        
        # Parse JSON (should be clean with structured output)
        predictions_dict = json.loads(raw_response)
        
        # Extract predictions in order
        results = []
        for i in range(len(batch_tokens_list)):
            key = str(i)
            if key in predictions_dict:
                pred = predictions_dict[key]
                if isinstance(pred, list) and all(isinstance(tag, str) for tag in pred):
                    results.append(pred)
                else:
                    results.append(f"Error: Invalid format for example {i}")
            else:
                results.append(f"Error: Missing key {i} in response")
        
        return results
        
    except json.JSONDecodeError as e:
        # If batch fails, return errors for all examples
        error_msg = f"Error: JSON parsing failed: {e}"
        return [error_msg] * len(batch_tokens_list)
    
    except Exception as e:
        # If API call fails, return errors for all examples
        error_msg = f"Error: API call failed: {e}"
        return [error_msg] * len(batch_tokens_list)

print("✓ Batch prediction function defined")

## 5. Helper Function to Process Dataset in Batches

In [ ]:
def process_dataset_in_batches(model, prompt, tokens_list, batch_size=10, delay=2.0):
    """
    Process entire dataset in batches with progress bar.
    
    Args:
        model: Gemini model
        prompt: System prompt
        tokens_list: List of all token sequences
        batch_size: Number of examples per API call (default 10)
        delay: Delay between API calls in seconds
    
    Returns:
        List of predictions for all examples
    """
    all_predictions = []
    total_batches = (len(tokens_list) + batch_size - 1) // batch_size
    
    print(f"Processing {len(tokens_list)} examples in {total_batches} batches (batch_size={batch_size})")
    print(f"Estimated time: ~{total_batches * delay / 60:.1f} minutes\n")
    
    # Process in batches with progress bar
    for i in tqdm(range(0, len(tokens_list), batch_size), desc="Processing batches"):
        batch = tokens_list[i:i+batch_size]
        batch_predictions = get_batch_predictions(model, prompt, batch, delay)
        all_predictions.extend(batch_predictions)
    
    # Count errors
    errors = sum(1 for pred in all_predictions if isinstance(pred, str) and pred.startswith("Error"))
    valid = len(all_predictions) - errors
    
    print(f"\n✓ Complete! Valid: {valid}, Errors: {errors}")
    
    return all_predictions

print("✓ Batch processing function defined")

## 6. Load Data

In [ ]:
# Load validation split
try:
    val_data = pd.read_json('val_split.jsonl', lines=True)
    print(f"✓ Loaded val_split.jsonl: {len(val_data)} examples")
    print(f"  Columns: {list(val_data.columns)}")
    display(val_data.head(3))
except FileNotFoundError:
    print("✗ Error: val_split.jsonl not found")
    val_data = pd.DataFrame()
except Exception as e:
    print(f"✗ Error loading val_split.jsonl: {e}")
    val_data = pd.DataFrame()

In [ ]:
# Load test data
try:
    test_data = pd.read_json('test_data.jsonl', lines=True)
    print(f"✓ Loaded test_data.jsonl: {len(test_data)} examples")
    print(f"  Columns: {list(test_data.columns)}")
    display(test_data.head(3))
except FileNotFoundError:
    print("✗ Error: test_data.jsonl not found")
    test_data = pd.DataFrame()
except Exception as e:
    print(f"✗ Error loading test_data.jsonl: {e}")
    test_data = pd.DataFrame()

## 7. Generate Predictions on Validation Set (Batched)

**Configuration**:
- Batch size: 10 examples per API call
- Delay: 2 seconds between calls
- Using full detailed prompt for robustness
- Expected time: ~2 hours for 10,000 examples
- Expected cost: ~$80-100

**vs Original (single example)**:
- Original: 16+ hours, $574
- Optimized with batching: 2 hours, $90 (8x faster, 6x cheaper!)

In [ ]:
if not val_data.empty and 'tokens' in val_data.columns:
    # Configuration
    BATCH_SIZE = 10  # Adjust if needed (5-20 recommended)
    DELAY = 2.0      # Adjust if hitting rate limits
    
    print("Starting batch prediction on validation set...")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Delay: {DELAY}s per batch\n")
    
    # Process in batches
    val_data['predicted_tags'] = process_dataset_in_batches(
        gemini_model,
        FULL_SYSTEM_PROMPT,
        val_data['tokens'].tolist(),
        batch_size=BATCH_SIZE,
        delay=DELAY
    )
    
    print("\n✓ Predictions complete!")
    display(val_data[['id', 'tokens', 'ner_tags', 'predicted_tags']].head())
else:
    print("✗ Cannot generate predictions: val_data is empty or missing 'tokens' column")

## 8. Save Validation Predictions

In [ ]:
if not val_data.empty and 'predicted_tags' in val_data.columns:
    output_file = 'val_split_predictions_batch_fullprompt.jsonl'
    val_data.to_json(output_file, orient='records', lines=True)
    print(f"✓ Validation predictions saved to {output_file}")
    print(f"  Total examples: {len(val_data)}")
    
    # Count valid vs errors
    valid = val_data['predicted_tags'].apply(lambda x: isinstance(x, list)).sum()
    errors = len(val_data) - valid
    print(f"  Valid predictions: {valid}")
    print(f"  Errors: {errors}")
else:
    print("✗ No predictions to save")

## 9. Evaluate on Validation Set

In [ ]:
try:
    import utils
    print("✓ utils.py imported successfully")
    
    # Load predictions
    predictions_df = pd.read_json('val_split_predictions_batch_fullprompt.jsonl', lines=True)
    
    # Filter out error messages (keep only valid predictions)
    valid_df = predictions_df[predictions_df['predicted_tags'].apply(lambda x: isinstance(x, list))]
    
    if len(valid_df) > 0:
        ground_truth = valid_df['ner_tags'].tolist()
        predicted = valid_df['predicted_tags'].tolist()
        tokens = valid_df['tokens'].tolist()
        
        print(f"\nEvaluating {len(valid_df)} valid predictions...")
        print(f"Skipped {len(predictions_df) - len(valid_df)} errors\n")
        
        # Evaluate
        results = utils.evaluate_entity_spans(ground_truth, predicted, tokens)
        
        print("\n" + "="*80)
        print("EVALUATION RESULTS - Gemini Few-Shot (Batched + Full Prompt)")
        print("="*80)
        print(f"Precision: {results['precision']:.4f} ({results['precision']*100:.2f}%)")
        print(f"Recall:    {results['recall']:.4f} ({results['recall']*100:.2f}%)")
        print(f"F1 Score:  {results['f1']:.4f} ({results['f1']*100:.2f}%)")
        print(f"\nTrue Positives:  {results['true_positives']}")
        print(f"False Positives: {results['false_positives']}")
        print(f"False Negatives: {results['false_negatives']}")
        print("="*80)
        
        # Detailed report
        print("\n")
        utils.print_evaluation_report(ground_truth, predicted, tokens,
                                     model_name="Gemini 2.5 Flash (Batched + Full Prompt)")
    else:
        print("✗ No valid predictions to evaluate")
        
except FileNotFoundError as e:
    print(f"✗ File not found: {e}")
    print("  Make sure predictions were saved successfully.")
except ImportError:
    print("✗ Error: utils.py not found")
    print("  Make sure utils.py is uploaded to Colab.")
except Exception as e:
    print(f"✗ Error during evaluation: {e}")

## 10. Generate Predictions on Test Set (Batched)

In [ ]:
if not test_data.empty and 'tokens' in test_data.columns:
    # Configuration (same as validation)
    BATCH_SIZE = 10
    DELAY = 2.0
    
    print("Starting batch prediction on test set...")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Delay: {DELAY}s per batch\n")
    
    # Process in batches
    test_data['predicted_tags'] = process_dataset_in_batches(
        gemini_model,
        FULL_SYSTEM_PROMPT,
        test_data['tokens'].tolist(),
        batch_size=BATCH_SIZE,
        delay=DELAY
    )
    
    print("\n✓ Predictions complete!")
    display(test_data[['id', 'tokens', 'predicted_tags']].head())
else:
    print("✗ Cannot generate predictions: test_data is empty or missing 'tokens' column")

## 11. Save Test Predictions

In [ ]:
if not test_data.empty and 'predicted_tags' in test_data.columns:
    output_file = 'test_data_predictions_batch_fullprompt.jsonl'
    test_data.to_json(output_file, orient='records', lines=True)
    print(f"✓ Test predictions saved to {output_file}")
    print(f"  Total examples: {len(test_data)}")
    
    # Count valid vs errors
    valid = test_data['predicted_tags'].apply(lambda x: isinstance(x, list)).sum()
    errors = len(test_data) - valid
    print(f"  Valid predictions: {valid}")
    print(f"  Errors: {errors}")
else:
    print("✗ No predictions to save")

## Summary

### Configuration:
✅ **Batch Processing**: 10 examples per API call  
✅ **Structured JSON Output**: Eliminates parsing errors  
✅ **Full Detailed Prompt**: Your original 2,500 token robust prompt  

### Performance vs Original:
| Metric | Original (Single) | This Version (Batched) | Improvement |
|--------|-------------------|------------------------|-------------|
| Time (10K examples) | 16.7 hours | ~2 hours | **8x faster** |
| Cost (10K examples) | $574 | ~$90 | **6x cheaper** |
| Reliability | Manual JSON parsing | Structured output | **More reliable** |
| Prompt quality | Full detailed | Same full detailed | **Same robustness** |

### Files Generated:
- `val_split_predictions_batch_fullprompt.jsonl` - Validation with predictions
- `test_data_predictions_batch_fullprompt.jsonl` - Test with predictions

### Trade-offs vs Shortened Prompt:
| Aspect | This (Full Prompt) | Shortened Prompt Version |
|--------|-------------------|-------------------------|
| Robustness | ✅ Better | ⚠️ Good |
| Cost | ~$90 | ~$45 |
| Time | ~2 hours | ~1.5 hours |
| Accuracy | Likely +1-2% F1 | Baseline |

### Next Steps:
1. Review validation F1 score
2. Compare with traditional models (M4 v2: 75.94%)
3. If F1 ≥ 75%, consider submitting
4. For best results: Train M8 (RoBERTa) on GPU → 85-88% F1

### Expected Performance:
- Few-shot with full prompt: **65-72% F1** (better than short prompt by ~2-3%)
- Your M4 v2: 75.94% F1
- Friends' BERT: 77-79% F1
- M8/M9 on GPU: 85-88% F1 (recommended for competition)
- State-of-the-art: 85%+ F1